<a href="https://colab.research.google.com/github/ramkumardeepak774/Text_Classification_Flair-vs-Fasttext/blob/main/text_classification_using_flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.getcwd())

/content


In [4]:
from google.colab import files
email_train = files.upload()
email_test = files.upload()

Saving idfc-email-train.csv to idfc-email-train.csv


Saving idfc-email-test.csv to idfc-email-test.csv


In [55]:
import io
train = pd.read_csv(io.BytesIO(email_train["idfc-email-train.csv"]))
test = pd.read_csv(io.BytesIO(email_test["idfc-email-test.csv"]))

In [57]:
train.columns  = ['label','text']
test.columns  = ['label','text']


In [60]:

# Using sklearn to shuffle rows
from sklearn.utils import shuffle
train = shuffle(train)
test = shuffle(test)

In [61]:
train.head()

,label,text
269,Card Application Status,Required Closing Statement of Credit Card. Dea...
424,Card Application Status,credit card idfc dosn't replay me for my busin...
534,Debit Card Offers,"Account Number: 10052476484 On 27th October, I..."
447,Card Application Status,KYC PENDING CREEDIT CARD KYC ka hamne schedule...
1102,Matured Loan Closure,Fwd: Regarding noc of loan >> ?Dear sir/Madam>...


In [29]:
# Create A Folder for the csv
!mkdir -p data

In [62]:
train.to_csv("data/train.csv")
test.to_csv("data/test.csv")

In [10]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 56.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 82.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:0

In [50]:
from flair.datasets import ClassificationCorpus,CSVClassificationCorpus
from flair.data import Corpus

In [63]:
# Create Column Mapping to show which column is for label and text
column_name_map = {1:"label",2:"text"}

In [74]:
# Location for CSV
data_folder = '/content/data'

In [86]:
# Create Corpus For CSV
corpus_csv: Corpus = CSVClassificationCorpus(data_folder,
                                             column_name_map=column_name_map,
                                             skip_header=True,
                                             label_type='label',
                                             delimiter=',')

2023-05-10 10:08:54,956 Reading data from /content/data
2023-05-10 10:08:54,957 Train: /content/data/train.csv
2023-05-10 10:08:54,959 Dev: None
2023-05-10 10:08:54,960 Test: /content/data/test.csv


In [88]:
# Creating the Label Diction For CSV
label_type='label'
label_dict_csv = corpus_csv.make_label_dictionary(label_type= label_type)


2023-05-10 10:09:07,786 Computing label dictionary. Progress:


1296it [00:03, 360.46it/s]

2023-05-10 10:09:11,388 Dictionary created for label 'label' with 7 values: Debit Card Offers (seen 225 times), Card Application Status (seen 218 times), Application Rejected (seen 216 times), Other Bank Charges (seen 215 times), Matured Loan Closure (seen 211 times), Fund Transfer (seen 211 times)


In [16]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=9d7c6f1ed0e92564754c81d12f9ffa0345588fe478c73ae6b73266fc31abebc5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [90]:
from flair.embeddings import SentenceTransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

In [83]:
document_embeddings=SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')

In [91]:
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict_csv, multi_label=True, label_type="label")

In [92]:
trainer = ModelTrainer(classifier, corpus_csv)

In [93]:
trainer.train('/content/data',
              learning_rate=0.5,
              mini_batch_size=50,
              max_epochs=5)

2023-05-10 10:09:42,908 ----------------------------------------------------------------------------------------------------
2023-05-10 10:09:42,910 Model: "TextClassifier(
  (embeddings): SentenceTransformerDocumentEmbeddings(
    (model): SentenceTransformer(
      (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
      (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    )
  )
  (decoder): Linear(in_features=768, out_features=7, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
  (weights): None
  (weight_tensor) None
)"
2023-05-10 10:09:42,915 ----------------------------------------------------------------------------------------------------
2023-05-10 10:09:42,917 Corpu

100%|██████████| 3/3 [00:02<00:00,  1.24it/s]

2023-05-10 10:10:13,622 Evaluating as a multi-label problem: True
2023-05-10 10:10:13,642 DEV : loss 14.547367095947266 - f1-score (micro avg)  0.3292


2023-05-10 10:10:13,872 BAD EPOCHS (no improvement): 0
2023-05-10 10:10:13,873 saving best model
2023-05-10 10:10:15,249 ----------------------------------------------------------------------------------------------------
2023-05-10 10:10:16,866 epoch 2 - iter 2/26 - loss 8.58165192 - time (sec): 1.62 - samples/sec: 61.90 - lr: 0.500000
2023-05-10 10:10:18,948 epoch 2 - iter 4/26 - loss 7.65204918 - time (sec): 3.70 - samples/sec: 54.10 - lr: 0.500000
2023-05-10 10:10:21,483 epoch 2 - iter 6/26 - loss 7.20167709 - time (sec): 6.23 - samples/sec: 48.13 - lr: 0.500000
2023-05-10 10:10:23,861 epoch 2 - iter 8/26 - loss 7.32918949 - time (sec): 8.61 - samples/sec: 46.46 - lr: 0.500000
2023-05-10 10:10:25,593 epoch 2 - iter 10/26 - loss 7.50335159 - time (sec): 10.34 - samples/sec: 48.34 - lr: 0.500000
2023-05-10 10:10:27,275 epoch 2 - iter 12/26 - loss 7.56929588 - time (sec): 12.02 - samples/sec: 49.90 - lr: 0.500000
2023-05-10 10:10:28,853 epoch 2 - iter 14/26 - loss 7.61196819 - time (s

100%|██████████| 3/3 [00:02<00:00,  1.21it/s]

...

2023-05-10 10:12:14,294 BAD EPOCHS (no improvement): 0
2023-05-10 10:12:14,295 saving best model
2023-05-10 10:12:17,454 ----------------------------------------------------------------------------------------------------


100%|██████████| 8/8 [00:07<00:00,  1.05it/s]

2023-05-10 10:12:27,399 Evaluating as a multi-label problem: True


2023-05-10 10:12:27,423 0.5035	0.6056	0.5498	0.4083
2023-05-10 10:12:27,424 
Results:
- F-score (micro) 0.5498
- F-score (macro) 0.5133
- Accuracy 0.4083

By class:
                         precision    recall  f1-score   support

   Application Rejected     0.3455    0.9500    0.5067        60
          Fund Transfer     0.3852    0.7833    0.5165        60
     Other Bank Charges     0.6324    0.7167    0.6719        60
   Matured Loan Closure     0.9245    0.8167    0.8673        60
      Debit Card Offers     0.8800    0.3667    0.5176        60
Card Application Status     0.0000    0.0000    0.0000        60

              micro avg     0.5035    0.6056    0.5498       360
              macro avg     0.5279    0.6056    0.5133       360
           weighted avg     0.5279    0.6056    0.5133       360
            samples avg     0.5056    0.6056    0.5384       360

2023-05-10 10:12:27,426 ---------------------------------------------------------------------------------------------

{'test_score': 0.5498108448928122,
 'dev_score_history': [0.3292181069958848,
  0.4444444444444445,
  0.4392156862745098,
  0.48081841432225064,
  0.5723270440251572],
 'train_loss_history': [9.591980898821795,
  7.380925943822037,
  7.021438221872589,
  6.337335386393983,
  6.708341380696238],
 'dev_loss_history': [14.547367095947266,
  11.749005317687988,
  7.99633264541626,
  14.232457160949707,
  5.957902431488037]}

text classfication on sysco data 

In [84]:
from google.colab import files
sysco = files.upload()


Saving sysco-full.csv to sysco-full.csv


In [ ]:
sysco.head()